In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

c:\Users\zhang\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Pre-processing Stage

In [2]:
no = pd.read_csv('no_discount.csv')
all = pd.read_csv('all_discount.csv')
half = pd.read_csv('half_discount.csv')
prod = pd.read_csv('product.csv')

## Data Preparation

In [3]:
# Map user features to number
dataframes = {'all': all, 'half': half, 'no': no}
feature_cols = ['user_feature_1', 'user_feature_2', 'user_feature_3']
category_maps = {}

# 1. Build mapping for each feature across all dataframes
for col in feature_cols:
    # Collect unique values from all dataframes for this column
    unique_vals = pd.concat([df[col] for df in dataframes.values()]).unique()
    cat_map = {cat: i+1 for i, cat in enumerate(sorted(unique_vals))}
    category_maps[col] = cat_map

# 2. Apply the mapping to each dataframe
for df in dataframes.values():
    for col in feature_cols:
        df[col + '_num'] = df[col].map(category_maps[col])

# 3. Show mapping for each feature
for col in feature_cols:
    print(f"\nMapping for {col}:")
    for cat, idx in category_maps[col].items():
        print(f"{idx}: {cat}")


Mapping for user_feature_1:
1: Female
2: Male
3: Non-binary / third gender
4: Prefer not to say

Mapping for user_feature_2:
1: 1-2 years ago
2: 2-3 years ago
3: 3-4 yeas ago
4: Currently, I don't have a phone
5: Less than 1 year ago
6: More than 4 years ago

Mapping for user_feature_3:
1: Brand reputation
2: Design and appearance
3: Good price
4: Product quality


In [4]:
# Map choice to number (no discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $24 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $26 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $27 (MSRP: $27)": 6
}

# Apply the mapping
no['choice_num'] = no['choice'].map(choice_map)

In [5]:
# Map choice to number (all discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $5.2 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $5 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $5 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
all['choice_num'] = all['choice'].map(choice_map)

In [6]:
# Map choice to number (half discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
half['choice_num'] = half['choice'].map(choice_map)

In [7]:
# Normalization:  price
# Define the columns to scale
columns_to_scale = [
    'price'
]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
prod[columns_to_scale] = scaler.fit_transform(prod[columns_to_scale])

In [8]:
# Set parameters
discount_percentage = 0.2
num_product = 6

In [9]:
# Check the number of valid data
num_row_no = len(no)
num_row_all = len(all)
num_row_half = len(half)
print(num_row_no)
print(num_row_all)
print(num_row_half)

291
278
266


# Analysis Stage

## True GTE

In [10]:
# All products are control (no discount)
# Merge the purchasing data with the product data
merged_no = no.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_control = 0
revenue_control = merged_no['price'].sum() * 300/291

# All products are treated (all discount)
# Merge the purchasing data with the product data
merged_all = all.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_treated = 0
revenue_treated = merged_all['price'].sum() * discount_percentage * 300/278

# Calculate the GTE
true = revenue_treated - revenue_control
print(true)

-68.6551789496238


## DIM Estimator

In [11]:
# Define the function to calculate the revenue
def calculate_total_revenue(df):
    revenue_treated = 0.0
    revenue_control = 0.0
    df['adjusted_price'] = df.apply(
        lambda row: row['price'] * discount_percentage if row['if_treated'] == 1 else row['price'], axis=1
    )
    for _, row in df.iterrows():
        if row['if_treated'] == 0:
            revenue_control += row['adjusted_price']
        elif row['if_treated'] == 1:
            revenue_treated += row['adjusted_price']
    return revenue_control, revenue_treated

# Merge the purchasing data with the product data
merged_half = half.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the revenue difference
revenue_control, revenue_treated = calculate_total_revenue(merged_half)

# Calculate the GTE
naive = (revenue_treated - revenue_control) * 600/266
naive
naive_pe = abs((naive - true)/true)
print(naive_pe)

0.11024849998145407


In [12]:
feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
encoded_features = pd.get_dummies(prod[feature_cols], dtype=float)
X_product = torch.tensor(encoded_features.iloc[1:].to_numpy(), dtype=torch.float)

user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
encoded_features = pd.get_dummies(merged_half[user_num_cols], dtype=float)
X_user = torch.tensor(encoded_features.to_numpy(), dtype=torch.float)

## MNL Estimator

In [13]:
# # Product features (skip the first row)
# feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
# X_product = torch.tensor(prod[feature_cols].iloc[1:].to_numpy(), dtype=torch.float)

# Price (skip first row)
price = torch.tensor(prod['price'].iloc[1:].to_numpy(), dtype=torch.float)

# Product randomization (skip first row)
prod_randomization = torch.tensor(prod['if_treated'].iloc[1:].to_numpy(), dtype=torch.bool)

# User features
user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
X_user = torch.tensor(merged_half[user_num_cols].to_numpy(),dtype=torch.float)

# User choice
choice = torch.tensor(
    merged_half['choice_num'].to_numpy(),
    dtype=torch.long
)

In [14]:
# Split the training and test set
X_user_train, X_user_test, choice_train, choice_test = train_test_split(
    X_user, choice, test_size=0.5, random_state=42
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_user_train = X_user_train.to(device)
X_user_test  = X_user_test.to(device)
X_product    = X_product.to(device)
price        = price.to(device)
prod_randomization = prod_randomization.to(device)

choice_train = choice_train.to(device)
choice_test  = choice_test.to(device)

# Validation set
X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train,test_size=0.1,random_state=34)

### No features

In [15]:
class LinearMNLModel_No(nn.Module):
    def __init__(self):
        super(LinearMNLModel_No, self).__init__()
        # v is the same constant value for all products.
        self.v = nn.Parameter(torch.tensor(1.0))
        # gamma is the price sensitivity parameter.
        self.gamma = nn.Parameter(torch.tensor(1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N = x_user.shape[0]
        # Adjust prices for treated products (discounted)
        adjusted_price = torch.where(
            prod_randomization,
            price * discount_percentage,
            price
        )
        # Compute utilities: shape [M]
        utility = self.v + self.gamma * adjusted_price
        # Expand to shape [N, M]
        utility = utility.expand(N, -1)
        # Add outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        return utilities_with_outside

In [16]:
model = LinearMNLModel_No().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.1) 

num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        # torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 2.959160566329956, Validation Loss: 2.8459665775299072
Epoch 101, Training Loss: 1.2196589708328247, Validation Loss: 1.1565020084381104
Epoch 201, Training Loss: 1.2172282934188843, Validation Loss: 1.133847713470459
Epoch 301, Training Loss: 1.2172260284423828, Validation Loss: 1.1332342624664307
Epoch 401, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948
Epoch 501, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948
Epoch 601, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948
Epoch 701, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948
Epoch 801, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948
Epoch 901, Training Loss: 1.2172260284423828, Validation Loss: 1.1332358121871948


In [17]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266 # For comparison purpose
print(linear)
linear_no = abs((linear - true)/true)
print(linear_no)

Expected Revenue: $37.51
Expected Revenue: $8.90
-64.52994669290413
0.06008625015378063


### Only user features

In [18]:
class LinearMNLModel_UserOnly(nn.Module):
    def __init__(self, user_feature_dim):
        super(LinearMNLModel_UserOnly, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user features
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_user + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [19]:
# Model training
model = LinearMNLModel_UserOnly(user_feature_dim=X_user.shape[1]).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 5.740757465362549, Validation Loss: 6.385035037994385
Epoch 101, Training Loss: 1.5111818313598633, Validation Loss: 1.6648344993591309
Epoch 201, Training Loss: 1.3114057779312134, Validation Loss: 1.4550843238830566
Epoch 301, Training Loss: 1.260339379310608, Validation Loss: 1.3194432258605957
Epoch 401, Training Loss: 1.226991057395935, Validation Loss: 1.213325023651123
Epoch 501, Training Loss: 1.2106239795684814, Validation Loss: 1.1434248685836792
Epoch 601, Training Loss: 1.2042226791381836, Validation Loss: 1.1021337509155273
Epoch 701, Training Loss: 1.2020015716552734, Validation Loss: 1.0792975425720215
Epoch 801, Training Loss: 1.2012255191802979, Validation Loss: 1.067275047302246
Epoch 901, Training Loss: 1.2009180784225464, Validation Loss: 1.0612200498580933


In [20]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_user_only = abs((linear - true)/true)
print(linear_user_only)

Expected Revenue: $36.43
Expected Revenue: $8.95
-61.987276005565676
0.09712163082337526


### Only product features

In [21]:
class LinearMNLModel_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim):
        super(LinearMNLModel_ProductOnly, self).__init__()
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand product features
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_product + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [22]:
# Model training
model = LinearMNLModel_ProductOnly(product_feature_dim=X_product.shape[1]).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 


num_epochs = 1000
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()


    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.4182103872299194, Validation Loss: 1.4443371295928955
Epoch 101, Training Loss: 1.1987248659133911, Validation Loss: 1.1952967643737793
Early stopping triggered at epoch 130


In [23]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_product_only = abs((linear - true)/true)
print(linear_product_only)

Expected Revenue: $30.96
Expected Revenue: $11.51
-43.865646993307244
0.3610730076824364


### All features

In [24]:
# MNL choice model
class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside

In [25]:
# Model training
model = LinearMNLModel(
    user_feature_dim=X_user.shape[1],
    product_feature_dim=X_product.shape[1]
).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 5000
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        # val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 4.151847839355469, Validation Loss: 3.3779635429382324
Epoch 101, Training Loss: 1.607783317565918, Validation Loss: 1.4097914695739746
Epoch 201, Training Loss: 1.3611581325531006, Validation Loss: 1.1588371992111206
Epoch 301, Training Loss: 1.2767632007598877, Validation Loss: 1.0957163572311401
Epoch 401, Training Loss: 1.2256914377212524, Validation Loss: 1.079655408859253
Epoch 501, Training Loss: 1.194152593612671, Validation Loss: 1.08278226852417
Epoch 601, Training Loss: 1.1777933835983276, Validation Loss: 1.096991777420044
Epoch 701, Training Loss: 1.1700944900512695, Validation Loss: 1.1149234771728516
Epoch 801, Training Loss: 1.1663105487823486, Validation Loss: 1.1321467161178589
Epoch 901, Training Loss: 1.1641597747802734, Validation Loss: 1.147223711013794
Epoch 1001, Training Loss: 1.1627538204193115, Validation Loss: 1.160097360610962
Early stopping triggered at epoch 1040


In [26]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_all = abs((linear - true)/true)
print(linear_all)

Expected Revenue: $30.40
Expected Revenue: $10.51
-44.87579460430862
0.34635965864663265


## PDL Estimator

In [27]:
# Prepare data
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    all_other_prices = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        other_product_prices = prices[indices]
        all_x_other_products.append(other_products)
        all_other_prices.append(other_product_prices)
    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
    all_other_prices = torch.stack(all_other_prices, dim=0)
    return user_features, product_features, prices, all_x_other_products, all_other_prices

### No features

In [28]:
class DeepMNLModel_No(nn.Module):
    """
    Price-only Deep MNL:
    u_ij = theta(p_j)
    Outside option utility = 0.
    """
    def __init__(self, num_product, hidden=5):
        super(DeepMNLModel_No, self).__init__()

        # theta(p_j):
        # Input:  [1]
        # Output: scalar utility
        self.theta = nn.Sequential(
        nn.Linear(1, hidden),
        nn.ReLU(),
        nn.Linear(hidden, 1)

)

    def forward(self, x_user, X_product, X_other_products, x_other_prices, price, prod_randomization):
        """
        Inputs used:
        - price: [M]  own price p_j
        - prod_randomization: [M]  treatment indicator
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # Apply treatment discount to own prices
        adjusted_price = torch.where(
            prod_randomization.to(device).bool(),
            price * discount_percentage,
            price
        )  # [M]

        # Prepare input for theta: [M, 1]
        pj_input = adjusted_price.unsqueeze(1)

        # Expand to all users: [N, M, 1]
        theta_input_exp = pj_input.unsqueeze(0).expand(N, -1, -1)

        # u_ij = theta(p_j): [N, M]
        utilities = self.theta(theta_input_exp).squeeze(-1)

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [29]:
# Apply the model
pdlmodel = DeepMNLModel_No(
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_No(
  (theta): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [30]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [31]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.7769, Validation Loss: 1.8092
Epoch 2, Training Loss: 1.7590, Validation Loss: 1.7921
Epoch 3, Training Loss: 1.7412, Validation Loss: 1.7752
Epoch 4, Training Loss: 1.7235, Validation Loss: 1.7583
Epoch 5, Training Loss: 1.7059, Validation Loss: 1.7416
Epoch 6, Training Loss: 1.6885, Validation Loss: 1.7251
Epoch 7, Training Loss: 1.6712, Validation Loss: 1.7088
Epoch 8, Training Loss: 1.6541, Validation Loss: 1.6927
Epoch 9, Training Loss: 1.6372, Validation Loss: 1.6769
Epoch 10, Training Loss: 1.6206, Validation Loss: 1.6613
Epoch 11, Training Loss: 1.6043, Validation Loss: 1.6460
Epoch 12, Training Loss: 1.5882, Validation Loss: 1.6310
Epoch 13, Training Loss: 1.5725, Validation Loss: 1.6164
Epoch 14, Training Loss: 1.5572, Validation Loss: 1.6022
Epoch 15, Training Loss: 1.5423, Validation Loss: 1.5883
Epoch 16, Training Loss: 1.5278, Validation Loss: 1.5748
Epoch 17, Training Loss: 1.5137, Validation Loss: 1.5617
Epoch 18, Training Loss: 1.5000, Validat

In [32]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_no = abs((pdl - true) / true)
print(pdl_no)


Expected Revenue (all control): $37.61
Expected Revenue (all treated): $8.85
-64.87762336444138
0.055022150447735826


### Only user features

In [147]:
class DeepMNLModel_UserOnly(nn.Module):
    """
    Deep MNL (user + own price only):
    u_ij = theta(x_i, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, num_product, hidden=5):
        super(DeepMNLModel_UserOnly, self).__init__()

        # theta(x_i, p_j)
        # Input per (i,j): [user_dim + 1]
        self.theta = nn.Sequential(
            nn.Linear(user_dim + 1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):

        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)          # [M, 1]

        pj_feat_exp = pj_feat.unsqueeze(0).expand(N, -1, -1)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        theta_input = torch.cat([x_user_exp, pj_feat_exp], dim=2)

        theta_input_flat = theta_input.reshape(N * M, -1)

        # u_ij: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside


In [148]:
# Apply the model
pdlmodel = DeepMNLModel_UserOnly(
    user_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# # Weight initialization
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserOnly(
  (theta): Sequential(
    (0): Linear(in_features=4, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [149]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [150]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 20
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.4532, Validation Loss: 2.4765
Epoch 2, Training Loss: 2.3600, Validation Loss: 2.3780
Epoch 3, Training Loss: 2.2770, Validation Loss: 2.2878
Epoch 4, Training Loss: 2.2045, Validation Loss: 2.2105
Epoch 5, Training Loss: 2.1416, Validation Loss: 2.1461
Epoch 6, Training Loss: 2.0868, Validation Loss: 2.0929
Epoch 7, Training Loss: 2.0415, Validation Loss: 2.0495
Epoch 8, Training Loss: 2.0049, Validation Loss: 2.0114
Epoch 9, Training Loss: 1.9759, Validation Loss: 1.9793
Epoch 10, Training Loss: 1.9515, Validation Loss: 1.9517
Epoch 11, Training Loss: 1.9321, Validation Loss: 1.9275
Epoch 12, Training Loss: 1.9151, Validation Loss: 1.9070
Epoch 13, Training Loss: 1.9026, Validation Loss: 1.8916
Epoch 14, Training Loss: 1.8930, Validation Loss: 1.8814
Epoch 15, Training Loss: 1.8852, Validation Loss: 1.8737
Epoch 16, Training Loss: 1.8784, Validation Loss: 1.8663
Epoch 17, Training Loss: 1.8718, Validation Loss: 1.8592
Epoch 18, Training Loss: 1.8655, Validat

In [151]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_user = abs((pdl - true) / true)
print(pdl_user)

Expected Revenue (all control): $37.05
Expected Revenue (all treated): $8.46
-64.49294269533085
0.060625233492538386


### Only product features

In [152]:
class DeepMNLModel_ProductOnly(nn.Module):
    """
    Deep MNL (product-only):
    u_ij = theta(z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, product_dim, num_product, hidden=5):
        super(DeepMNLModel_ProductOnly, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(z_j, z_-j, p_j)
        # Input per (i,j): [F_p + hidden (z_-j) + 1 (p_j)]
        in_dim = product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        X_product:        [M, F_p]          product features z_j
        X_other_products: [M, (M-1)*F_p]    other products' features z_-j
        price:            [M]               own price p_j (already discounted)
        """
        device = price.device
        # N users, M products
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j scalar: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Combine product-side info: [M, F_p + hidden + 1]
        product_feat = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand to all users: [N, M, in_dim]
        product_feat_exp = product_feat.unsqueeze(0).expand(N, -1, -1)

        # Flatten to [N*M, in_dim]
        theta_input_flat = product_feat_exp.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [157]:
# Apply the model
pdlmodel = DeepMNLModel_ProductOnly(
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# # Weight initialization
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [158]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [159]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 20
patience_counter = 0

# ----- Training loop -----
for epoch in range(1000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.2698, Validation Loss: 1.0990
Epoch 2, Training Loss: 1.2440, Validation Loss: 1.0921
Epoch 3, Training Loss: 1.2440, Validation Loss: 1.0903
Epoch 4, Training Loss: 1.2458, Validation Loss: 1.0870
Epoch 5, Training Loss: 1.2398, Validation Loss: 1.0790
Epoch 6, Training Loss: 1.2374, Validation Loss: 1.0775
Epoch 7, Training Loss: 1.2350, Validation Loss: 1.0806
Epoch 8, Training Loss: 1.2323, Validation Loss: 1.0849
Epoch 9, Training Loss: 1.2306, Validation Loss: 1.0893
Epoch 10, Training Loss: 1.2303, Validation Loss: 1.0932
Epoch 11, Training Loss: 1.2303, Validation Loss: 1.0960
Epoch 12, Training Loss: 1.2300, Validation Loss: 1.0973
Epoch 13, Training Loss: 1.2292, Validation Loss: 1.0969
Epoch 14, Training Loss: 1.2277, Validation Loss: 1.0954
Epoch 15, Training Loss: 1.2257, Validation Loss: 1.0932
Epoch 16, Training Loss: 1.2235, Validation Loss: 1.0908
Epoch 17, Training Loss: 1.2215, Validation Loss: 1.0887
Epoch 18, Training Loss: 1.2197, Validat

In [160]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_product = abs((pdl - true) / true)
print(pdl_product)

Expected Revenue (all control): $37.91
Expected Revenue (all treated): $9.36
-64.3806271086958
0.0622611710627756


### All features

In [169]:
# PDL choice model
class DeepMNLModel_UserProduct(nn.Module):
    """
    Deep MNL:
    u_ij = theta(x_i, z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, product_dim, num_product, hidden=5):
        super(DeepMNLModel_UserProduct, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(x_i, z_j, z_-j, p_j)
        # Input per (i,j):
        #   user_dim (x_i)
        # + product_dim (z_j)
        # + hidden (z_-j)
        # + 1 (p_j)
        in_dim = user_dim + product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user:          [N, F_u]          user features x_i
        X_product:       [M, F_p]          product features z_j
        X_other_products:[M, (M-1)*F_p]    other products' features z_-j
        price:           [M]               own price p_j (already discounted)

        x_other_prices, prod_randomization are not used here.
        """
        device = price.device
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Product-side block: [M, F_p + hidden + 1]
        product_block = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand product block for all users: [N, M, *]
        product_block_exp = product_block.unsqueeze(0).expand(N, -1, -1)

        # Expand user features: [N, 1, F_u] -> [N, M, F_u]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # Final theta input: [N, M, in_dim]
        theta_input = torch.cat([x_user_exp, product_block_exp], dim=2)

        # Flatten to [N*M, in_dim]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [170]:
# Apply the model
pdlmodel = DeepMNLModel_UserProduct(
    user_dim=X_user.shape[1],
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# # Weight initialization
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserProduct(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [171]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [172]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0 # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 50
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.7447, Validation Loss: 1.6316
Epoch 2, Training Loss: 1.6764, Validation Loss: 1.5406
Epoch 3, Training Loss: 1.5972, Validation Loss: 1.4349
Epoch 4, Training Loss: 1.5091, Validation Loss: 1.3334
Epoch 5, Training Loss: 1.4239, Validation Loss: 1.2394
Epoch 6, Training Loss: 1.3548, Validation Loss: 1.1501
Epoch 7, Training Loss: 1.3074, Validation Loss: 1.0815
Epoch 8, Training Loss: 1.2951, Validation Loss: 1.0496
Epoch 9, Training Loss: 1.3129, Validation Loss: 1.0517
Epoch 10, Training Loss: 1.3388, Validation Loss: 1.0574
Epoch 11, Training Loss: 1.3526, Validation Loss: 1.0569
Epoch 12, Training Loss: 1.3482, Validation Loss: 1.0511
Epoch 13, Training Loss: 1.3300, Validation Loss: 1.0445
Epoch 14, Training Loss: 1.3063, Validation Loss: 1.0415
Epoch 15, Training Loss: 1.2857, Validation Loss: 1.0438
Epoch 16, Training Loss: 1.2727, Validation Loss: 1.0515
Epoch 17, Training Loss: 1.2661, Validation Loss: 1.0625
Epoch 18, Training Loss: 1.2652, Validat

In [173]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_all = abs((pdl - true) / true)
print(pdl_all)

Expected Revenue (all control): $37.59
Expected Revenue (all treated): $8.76
-65.0320411624765
0.05277297128313949


## DML Estimator

In [174]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = num_product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    treated_price_with_outside = torch.cat((torch.zeros(1, device=all_treated_price.device), all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities * treated_price_with_outside - all_control_probabilities * price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside,dim=1)
    H_theta1 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside * price_with_outside,dim=1)

    return H,H_theta0,H_theta1

In [175]:
# l_theta
def l_theta(theta0_output, theta1_output, adjusted_price, decision_test, l2_lambda=1e-4):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]

    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price

    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti_with_outside = torch.cat((zero_utilities, uti), dim=1)

    probabilities = F.softmax(uti_with_outside, dim=1)

    prod_indices = torch.ones(M, device=uti.device)
    prod_indices = torch.cat([torch.zeros(1, device=uti.device), prod_indices])

    adjusted_price_with_outside = torch.cat([torch.zeros(1, device=adjusted_price.device), adjusted_price])

    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] - prod_indices[decision_test]

    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] * adjusted_price_with_outside[decision_test]) - \
              (adjusted_price_with_outside[decision_test] * prod_indices[decision_test])

    is_inside = (decision_test > 0)

    inside_indices = (decision_test - 1).clamp(min=0)

    theta0_chosen = theta0_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)
    theta1_chosen = theta1_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)

    reg_grad0 = 2 * l2_lambda * theta0_chosen
    reg_grad1 = 2 * l2_lambda * theta1_chosen

    ltheta0 = ltheta0 + (reg_grad0 * is_inside.float())
    ltheta1 = ltheta1 + (reg_grad1 * is_inside.float())

    return ltheta0, ltheta1

In [176]:
# l_inv
def lambdainv(theta0_output, theta1_output, price, decision_test, epsilon=1e-6, l2_lambda=1e-4):

    N = theta0_output.shape[0]
    M = num_product

    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount_percentage * price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti = theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    all_treated_price_vec = price * discount_percentage # Assuming logic from original code
    treated_price_with_outside = torch.cat((torch.zeros(1, device=price.device), all_treated_price_vec), dim=0)

    expand_price_ext = price_with_outside.unsqueeze(0).expand(N, M+1)

    chosen_price = expand_price_ext[torch.arange(N), decision_test]

    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + \
               chosen_prob_treated * (1 - chosen_prob_treated)

    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * chosen_price + \
               chosen_prob_treated * (1 - chosen_prob_treated) * (chosen_price * discount_percentage)

    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * (chosen_price**2) + \
               chosen_prob_treated * (1 - chosen_prob_treated) * ((chosen_price * discount_percentage)**2)

    ltheta00 = ltheta00 / 2
    ltheta01 = ltheta01 / 2
    ltheta11 = ltheta11 / 2

    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1) # [N, 2, 2]

    reg_hessian_value = 2 * l2_lambda
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device)
    reg_matrix = identity_matrix.unsqueeze(0).expand(N, 2, 2) * reg_hessian_value

    L_total = L_matrix + reg_matrix + (identity_matrix.unsqueeze(0) * epsilon)

    L_inv = torch.linalg.inv(L_total)

    return L_inv

### No features

In [177]:
class UtilityEstimator_No(nn.Module):
    def __init__(self, num_product): 
        super(UtilityEstimator_No, self).__init__()
        self.num_product = num_product

        self.theta0 = nn.Parameter(torch.tensor(1.0)) 
        self.theta1 = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        price: [M]
        """
        N = x_user.shape[0]
        M = self.num_product

        u_product = self.theta0 + self.theta1 * price 
        
        utilities = u_product.unsqueeze(0).expand(N, -1)
        zero_utilities = torch.zeros(N, 1, device=price.device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)


        theta0_expanded = self.theta0.view(1, 1).expand(N, M)
        theta1_expanded = self.theta1.view(1, 1).expand(N, M)

        return utilities_with_outside, theta0_expanded, theta1_expanded

In [178]:
# Apply the model
dml_model = UtilityEstimator_No(
    num_product=X_product.shape[0]
).to(device)


# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
dml_model.apply(init_weights)

UtilityEstimator_No()

In [179]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [180]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(1000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.2338318824768066, Validation Loss: 2.309324026107788
Epoch 2, Training Loss: 2.225825071334839, Validation Loss: 2.3013405799865723
Epoch 3, Training Loss: 2.217846393585205, Validation Loss: 2.2933857440948486
Epoch 4, Training Loss: 2.209897518157959, Validation Loss: 2.28546142578125
Epoch 5, Training Loss: 2.2019786834716797, Validation Loss: 2.2775678634643555
Epoch 6, Training Loss: 2.1940906047821045, Validation Loss: 2.2697055339813232
Epoch 7, Training Loss: 2.186234474182129, Validation Loss: 2.2618749141693115
Epoch 8, Training Loss: 2.178410053253174, Validation Loss: 2.2540762424468994
Epoch 9, Training Loss: 2.1706180572509766, Validation Loss: 2.2463104724884033
Epoch 10, Training Loss: 2.1628592014312744, Validation Loss: 2.2385780811309814
Epoch 11, Training Loss: 2.1551339626312256, Validation Loss: 2.230879545211792
Epoch 12, Training Loss: 2.1474430561065674, Validation Loss: 2.2232155799865723
Epoch 13, Training Loss: 2.139787197113037, Va

In [181]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [182]:
H,H_theta0,H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 1]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
sdl_no = abs((sdl - true) / true)
print(sdl_no)
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_no = abs((dedl - true) / true)
print(dedl_no)

0.09822353560522813
-71.10189280115573
0.0356377171972302


### Only user features


In [183]:
class UtilityEstimator_UserOnly(nn.Module):
    def __init__(self, user_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_UserOnly, self).__init__()

        # theta0(x_i)
        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(x_i)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]
        price:            [M]
        """
        device = x_user.device
        N = x_user.shape[0]
        M = price.shape[0]

        # N, Fu] -> [N, M, Fu]
        xi_exp = x_user.unsqueeze(1).expand(-1, M, -1)   # [N, M, Fu]

        # theta0(x_i): [N, M, 1] -> [N, M]
        theta0_output = self.theta0(xi_exp).squeeze(-1)  # [N, M]

        # theta1(x_i): [N, M, 1] -> [N, M]
        theta1_output = self.theta1(xi_exp).squeeze(-1)  # [N, M]

        # [N, M]
        price_exp = price.view(1, M).expand(N, M)        # [N, M]

        # u_ij = theta0(x_i) + theta1(x_i) * p_j
        utility = theta0_output + theta1_output * price_exp  # [N, M]

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output


In [184]:
# Apply the model
dml_model = UtilityEstimator_UserOnly(
    user_feature_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_UserOnly(
  (theta0): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [185]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [186]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.614642858505249, Validation Loss: 1.3462623357772827
Epoch 2, Training Loss: 1.562799334526062, Validation Loss: 1.3094476461410522
Epoch 3, Training Loss: 1.5143229961395264, Validation Loss: 1.2756067514419556
Epoch 4, Training Loss: 1.470048189163208, Validation Loss: 1.2456281185150146
Epoch 5, Training Loss: 1.4301544427871704, Validation Loss: 1.218544363975525
Epoch 6, Training Loss: 1.3938353061676025, Validation Loss: 1.1942564249038696
Epoch 7, Training Loss: 1.3617098331451416, Validation Loss: 1.1730124950408936
Epoch 8, Training Loss: 1.3338265419006348, Validation Loss: 1.153957724571228
Epoch 9, Training Loss: 1.3099641799926758, Validation Loss: 1.1368231773376465
Epoch 10, Training Loss: 1.2899805307388306, Validation Loss: 1.1220753192901611
Epoch 11, Training Loss: 1.273714303970337, Validation Loss: 1.1097302436828613
Epoch 12, Training Loss: 1.2607635259628296, Validation Loss: 1.0997651815414429
Epoch 13, Training Loss: 1.2508301734924316

In [187]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [188]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon,  l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_user_only = abs((dedl - true) / true)
print(dedl_user_only)

-65.08987254666206
0.05193062573732137


### Only product features

In [189]:
class UtilityEstimator_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_ProductOnly, self).__init__()

        # Process other products' features z_-j: input [M, (M-1)*Fp] -> output [M, Fp]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = product_feature_dim + product_feature_dim  # z_j + aggregated z_-j

        # theta0(z_j, aggregated z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(z_j, aggregated z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_product:        [M, Fp]
        x_other_products: [M, (M-1)*Fp]
        price:            [M]
        """
        device = x_product.device
        N = x_user.shape[0]
        M = x_product.shape[0]
        Fp = x_product.shape[1]

        # Aggregate z_-j: [M, (M-1)*Fp] -> [M, Fp]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, Fp]

        # Expand to [N, M, *]
        zj_exp     = x_product.unsqueeze(0).expand(N, -1, -1)                 # [N, M, Fp]
        zminus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1) # [N, M, Fp]

        # 拼成 theta 输入: [N, M, 2*Fp]
        feat_theta = torch.cat([zj_exp, zminus_exp], dim=2)                   # [N, M, 2*Fp]

        # theta0 输出: [N, M]
        theta0_output = self.theta0(feat_theta).squeeze(-1)

        # theta1 输出: [N, M]
        theta1_output = self.theta1(feat_theta).squeeze(-1)

        # Broadcast prices to [N, M]
        price_exp = price.view(1, M).expand(N, M)                             # [N, M]

        # Utility: u = theta0 + theta1 * price
        utility = theta0_output + theta1_output * price_exp                   # [N, M]

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [190]:
# Apply the model
dml_model = UtilityEstimator_ProductOnly(
    product_feature_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=3
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
  )
  (theta0): Sequential(
    (0): Linear(in_features=6, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=6, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
)

In [191]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [192]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.1491801738739014, Validation Loss: 2.0622847080230713
Epoch 2, Training Loss: 1.8937851190567017, Validation Loss: 1.8104190826416016
Epoch 3, Training Loss: 1.7100096940994263, Validation Loss: 1.6800473928451538
Epoch 4, Training Loss: 1.6350525617599487, Validation Loss: 1.6541050672531128
Epoch 5, Training Loss: 1.6072957515716553, Validation Loss: 1.6477246284484863
Epoch 6, Training Loss: 1.5864030122756958, Validation Loss: 1.653418779373169
Epoch 7, Training Loss: 1.5745365619659424, Validation Loss: 1.6601144075393677
Epoch 8, Training Loss: 1.5684996843338013, Validation Loss: 1.6657077074050903
Epoch 9, Training Loss: 1.5660704374313354, Validation Loss: 1.667614459991455
Epoch 10, Training Loss: 1.5589815378189087, Validation Loss: 1.661526083946228
Epoch 11, Training Loss: 1.5485984086990356, Validation Loss: 1.649787425994873
Epoch 12, Training Loss: 1.5392287969589233, Validation Loss: 1.6440749168395996
Epoch 13, Training Loss: 1.53392946720123

In [193]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [194]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_product_only = abs((dedl - true) / true)
print(dedl_product_only)

-66.3854642021925
0.03305962903536712


### All features

In [195]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()

        # Layers to process other products' features (z_-j)
        # Input:  [M, (M-1)*F_p] -> Output: [M, F_p]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 3),
            nn.ReLU(),
            nn.Linear(3,3),
            nn.ReLU(),
            nn.Linear(3, product_feature_dim)
        )

        in_dim = user_feature_dim + 2 * product_feature_dim  # x_i, z_j, z_-j

        # theta0(x_i, z_j, z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, 3),
            nn.ReLU(),
            nn.Linear(3, 3),
            nn.ReLU(),
            nn.Linear(3, 1)
        )

        # theta1(x_i, z_j, z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, 3),
            nn.ReLU(),
            nn.Linear(3, 3),
            nn.ReLU(),
            nn.Linear(3, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, F_u]
        x_product:        [M, F_p]
        x_other_products: [M, (M-1)*F_p]
        price:            [M]
        """
        N = x_user.shape[0]
        M = x_product.shape[0]

        # Process other products' features: [M, (M-1)*F_p] -> [M, F_p]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, F_p]

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)                    # [N, M, F_u]
        z_j_exp    = x_product.unsqueeze(0).expand(N, -1, -1)                # [N, M, F_p]
        z_minus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1)  # [N, M, F_p]

        # Features for theta0/theta1: [N, M, F_u + 2*F_p]
        combined_features = torch.cat(
            (x_user_exp, z_j_exp, z_minus_exp),
            dim=2
        )  # [N, M, in_dim]

        # Theta0, Theta1: [N, M, 1] -> [N, M]
        theta0_output = self.theta0(combined_features).squeeze(-1)  # [N, M]
        theta1_output = self.theta1(combined_features).squeeze(-1)  # [N, M]

        # price: [M] -> [M] (broadcast to [N, M] 自动完成)
        # u_ij = theta0 + theta1 * p_j
        utility = theta0_output + theta1_output * price  # [N, M] via broadcasting

        # Include the outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [196]:
0.363363*20,-3.409537*5,-6.272173*2,-15.913756/2


(7.26726, -17.047684999999998, -12.544346, -7.956878)

In [197]:
# Apply the model
dml_model = UtilityEstimator(user_feature_dim=X_user.shape[1],
                       product_feature_dim=X_product.shape[1]).to(device)

# # Initialize weight
# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             m.bias.data.fill_(0.01)

# dml_model.apply(init_weights)

In [198]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [199]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 20
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.2162094116210938, Validation Loss: 2.2323780059814453
Epoch 2, Training Loss: 2.207106590270996, Validation Loss: 2.2242302894592285
Epoch 3, Training Loss: 2.198281764984131, Validation Loss: 2.2166411876678467
Epoch 4, Training Loss: 2.189634084701538, Validation Loss: 2.208949327468872
Epoch 5, Training Loss: 2.1810719966888428, Validation Loss: 2.2015762329101562
Epoch 6, Training Loss: 2.172584056854248, Validation Loss: 2.1944892406463623
Epoch 7, Training Loss: 2.1640512943267822, Validation Loss: 2.187129259109497
Epoch 8, Training Loss: 2.1554079055786133, Validation Loss: 2.1793601512908936
Epoch 9, Training Loss: 2.146484375, Validation Loss: 2.171163558959961
Epoch 10, Training Loss: 2.1371257305145264, Validation Loss: 2.1626033782958984
Epoch 11, Training Loss: 2.1271049976348877, Validation Loss: 2.1525540351867676
Epoch 12, Training Loss: 2.115952968597412, Validation Loss: 2.1412296295166016
Epoch 13, Training Loss: 2.104374885559082, Validati

In [200]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [201]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
0.1,1, ]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_all = abs((dedl - true) / true)
print(dedl_all)
print(epsilon)

-71.18263818267593
0.0368138175694898
1


# Results

In [202]:
# Collect all results
results_data = {
    'Method': [
        'Naive',
        'Linear (No features)',
        'Linear (User only)',
        'Linear (Product only)',
        'Linear (All features)',
        'PDL (No features)',
        'PDL (User only)',
        'PDL (Product only)',
        'PDL (All features)',
        'DEDL (No features)',
        'DEDL (User only)',
        'DEDL (Product only)',
        'DEDL (All features)'
    ],
    'Abosulte Percentage Error': [
        naive_pe,
        linear_no,
        linear_user_only,
        linear_product_only,
        linear_all,
        pdl_no,
        pdl_user,
        pdl_product,
        pdl_all,
        dedl_no,
        dedl_user_only,
        dedl_product_only,
        dedl_all
    ]
}

# Create DataFrame
results_df = pd.DataFrame(results_data)

# Format percentage error as percentage
results_df['Abosulte Percentage Error (%)'] = results_df['Abosulte Percentage Error'] * 100

# Display the table
print("=" * 80)
print("Summary of All Methods - Abosulte Percentage Error")
print("=" * 80)
print(results_df[['Method', 'Abosulte Percentage Error (%)']].to_string(index=False))
print("=" * 80)

# Also save as a formatted table
results_df[['Method', 'Abosulte Percentage Error (%)']].round(4)


Summary of All Methods - Abosulte Percentage Error
               Method  Abosulte Percentage Error (%)
                Naive                      11.024850
 Linear (No features)                       6.008625
   Linear (User only)                       9.712163
Linear (Product only)                      36.107301
Linear (All features)                      34.635966
    PDL (No features)                       5.502215
      PDL (User only)                       6.062523
   PDL (Product only)                       6.226117
   PDL (All features)                       5.277297
   DEDL (No features)                       3.563772
     DEDL (User only)                       5.193063
  DEDL (Product only)                       3.305963
  DEDL (All features)                       3.681382


,Method,Abosulte Percentage Error (%)
0,Naive,11.0248
1,Linear (No features),6.0086
2,Linear (User only),9.7122
3,Linear (Product only),36.1073
4,Linear (All features),34.6360
5,PDL (No features),5.5022
6,PDL (User only),6.0625
7,PDL (Product only),6.2261
8,PDL (All features),5.2773
9,DEDL (No features),3.5638
